In [ ]:
import sys, os
sys.path
from os.path import join, abspath
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import tensor
import json
import random
import yaml
from addict import Dict
import ast
from utils import ExpandStrArrayColumns, MovingAvg

In [ ]:
def Dashboard(train_report: pd.DataFrame, eval_report: pd.DataFrame, window_size: int=100):
    loss_ma = MovingAvg(train_report.loss, window_size)
    dice_ma = MovingAvg(train_report.dice_scores, window_size)
    ce_ma = MovingAvg(train_report.cross_entropy_scores, window_size)
    sel_cols = ["learning_rate", "dice_scores", "cross_entropy_scores"]
    train_epoch_stats = train_report.groupby("epoch").mean(numeric_only=True)[sel_cols]
    eval_epoch_stats = eval_report.groupby("epoch").mean(numeric_only=True)
    lim = len(eval_epoch_stats.index)
    fig, axs = plt.subplots(2, 3, figsize=(12,8))
    fig.suptitle("Training Log")
    # plot loss
    axs[0, 0].set_title("Loss")
    axs[0, 0].set_xlabel("Block")
    axs[0, 0].set_ylabel("Loss")
    axs[0, 0].scatter(train_report.index, train_report.loss, s=0.3, c="red", label="value")
    axs[0, 0].plot(train_report.index, loss_ma, "-b", label="moving avg")
    axs[0, 0].legend()
    # plot block dice
    axs[0, 1].set_title("Block Dice")
    axs[0, 1].set_xlabel("Block")
    axs[0, 1].set_ylabel("Score")
    # axs[0, 1].set_ylim([-0.05, 1.05])
    axs[0, 1].scatter(train_report.index, train_report.dice_scores, s=0.5, c="red", label="dice_score")
    axs[0, 1].plot(train_report.index, dice_ma, "-b", label="moving avg")
    axs[0, 1].legend()
    # plot block ce
    axs[0, 2].set_title("Block Cross Entropy")
    axs[0, 2].set_xlabel("Block")
    axs[0, 2].set_ylabel("Score")
    axs[0, 2].scatter(train_report.index, train_report.cross_entropy_scores, s=0.5, c="blue", label="ce_score")
    axs[0, 2].plot(train_report.index, ce_ma, "-r", label="moving avg")
    axs[0, 2].legend()
    # plot learning rate
    axs[1, 0].set_title("Learning Rate")
    axs[1, 0].set_xlabel("Block")
    axs[1, 0].set_ylabel("Learning Rate")
    axs[1, 0].plot(train_report.index, train_report.learning_rate, "-r", linewidth=1.)
    # plot epoch dice
    axs[1, 1].set_title("Epoch Dice")
    axs[1, 1].set_xlabel("Epoch")
    axs[1, 1].set_ylabel("Score")
    # axs[1, 1].set_ylim([-0.05, 1.05])
    axs[1, 1].plot(eval_epoch_stats.index, eval_epoch_stats.dice_scores, "or",
                   linewidth=1.5, label="eval_dice")
    axs[1, 1].plot(eval_epoch_stats.index, train_epoch_stats.dice_scores[:lim], "ob",
                   linewidth=1.5, label="train_dice")
    axs[1, 1].legend()
    # plot epoch dice
    axs[1, 2].set_title("Epoch Cross Entropy")
    axs[1, 2].set_xlabel("Epoch")
    axs[1, 2].set_ylabel("Score")
    axs[1, 2].plot(eval_epoch_stats.index, eval_epoch_stats.cross_entropy_scores, "or",
                   linewidth=1.5, label="eval_ce")
    axs[1, 2].plot(eval_epoch_stats.index, train_epoch_stats.cross_entropy_scores[:lim], "ob",
                   linewidth=1.5, label="train_ce")
    axs[1, 2].legend()
    plt.tight_layout()
    plt.show() 

In [ ]:
train_rep_dir = "/home/isaiah/TotalSegmentator/results/20230330/nnunet_train_reps_01/"
eval_rep_dir = "/home/isaiah/TotalSegmentator/results/20230330/nnunet_eval_reps_01/"

tdfs = []
edfs = []
for i in range(5):
    tfile = join(train_rep_dir, f"rank{i}.csv")
    efile = join(eval_rep_dir, f"rank{i}.csv")
    tdfs.append(pd.read_csv(tfile))
    edfs.append(pd.read_csv(efile))
tdf = pd.concat(tdfs, axis=0)
edf = pd.concat(edfs, axis=0)
dfg_train = tdf.groupby("block").mean().astype({"epoch": np.uint16})
efg_train = ExpandStrArrayColumns(edf, "epoch")

In [ ]:
Dashboard(dfg_train, efg_train, window_size=100)